In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
paths = list((Path.cwd() / ".." / "experiments" / "calls-experiment").glob("*/*/*.df.gzip"))

df = pd.concat((pd.read_pickle(path) for path in paths), ignore_index=True)

In [ ]:
for (instance, fsg, multiplier), g1 in df.groupby(["instance", "forbidden_subgraphs", "multiplier"]):
    if len(g1) == 0 or np.isnan(g1["k"].str[-1].max()): continue
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.set_yscale("log")
    
    ax.grid(True)
    
    k_min, k_max = 0, int(np.ceil(g1["k"].str[-1].max() / multiplier))
    bins = np.linspace(k_min - 1, k_max + 1, max(2, min([k_max, 20])))
    assert len(bins) > 0, f"{k_min} {k_max}"
    
    lower_bounds = ["Trivial", "Greedy", "LocalSearch", "SortedGreedy"]
    labels = ["No lower bound", "Simple packing", "Local search", "Greedy lower bound"]
    for lower_bound, label in zip(lower_bounds, labels):
        g2 = g1[(g1["lower_bound"] == lower_bound) & (g1["selector"] == "MostAdjacentSubgraphs")]
        
        k, calls, time = zip(*[(
            np.array(row["k"]) / multiplier,
            np.array(row["calls"]),
            np.array(row["time"]) / 10**9) for _, row in g2.iterrows()])
        k, calls, time = map(np.concatenate, (k, calls, time))
        
        inds = np.digitize(k, bins)
        
        #sns.lineplot(bins[inds] - bins[1], time, ax=ax, label=label, legend=False)
        ax.scatter(k, calls, label=label, s=10)

    if g2["solution_cost"].max() >= 0:
        ax.axvline(x=g2["solution_cost"].max() / multiplier, c="black")
    
                   
    ax.set_xlabel("Editing cost $k$")
    #ax.set_ylabel("Time [s]")
    #ax.set_ylim((g1["time"].str[0].astype(float).min()  / 10**9, 10**1))
    ax.set_ylabel("Number of calls")
    ax.set_ylim((g1["calls"].str[0].astype(float).min(), g1["calls"].str[-1].astype(float).max()))

    print(f"{instance} {fsg}")
    ax.legend(loc="best")
    #fig.legend(loc="center left", bbox_to_anchor=(1, 0.5))
    fig.tight_layout()
    plt.savefig(f"calls/{instance}-{fsg}-calls.pdf")
    plt.show()